## Exploratory Data Analysis

#### This notebook contains steps to clean and inspect data before performing analyses. 

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
from mpl_toolkits.axes_grid.anchored_artists import AnchoredText
%matplotlib inline

In [ ]:
mpl.rcParams.update({
    'font.size'           : 16.0,
    'axes.titlesize'      : 'large',
    'axes.labelsize'      : 'medium',
    'xtick.labelsize'     : 'small',
    'ytick.labelsize'     : 'small',
    'legend.fontsize'     : 'small',
})

In [ ]:
# Read in data
raw = pd.read_csv('data/file.csv', parse_dates=[col])
raw.head()

In [ ]:
# Check the overall shape of the data (rows, columns)
raw.shape

In [ ]:
# Make changes to copy
df = raw.copy

In [ ]:
# Take a subsample if large data is running slowly
# df = df.iloc[:5000]
df = df.sample(frac=1, random_state=123)

In [ ]:
# Rename specific columns
df = df.rename(columns={'old name 1': 'new name’})
# Batch rename all columns
df.columns = ['']
# Clean up formatting for column names if needed
df.rename(columns=lambda x: x.strip().replace(" ", "_").lower(), inplace=True)

In [ ]:
# Checking data types of columns
df.info()

In [ ]:
# Change any data types that are incorrect
df.col = df.col.astype(data={col: dtype, ...}, copy=False)
# Check data type for all columns
df.dtypes

### Missing data
http://pandas.pydata.org/pandas-docs/stable/missing_data.html#cleaning-filling-missing-data

In [ ]:
# Boolean mask to see NaN values
pd.isnull(df)

In [ ]:
# Checking percentage of null values
nulls = df.isnull().sum()/float(df.shape[0])
nulls.sort_values(ascending=False)

In [ ]:
# Look at any found null values by column
null_values = df.loc[df[col].isnull()]
null_values

In [ ]:
# Checking percentage of null values in each row
null_rows = []
null_index = []

for i in range(5000):
    null_index.append(i)
    null_rows.append(df.iloc[i,:].isnull().sum()/float(df.shape[1]))
    
# Largest percent null values in rows
max(null_rows)

In [ ]:
# Drop columns with 25% or more null values
for col in df:
    if df[col].isnull().sum()/float(df.shape[0]) >= 0.25:
        df.drop(col, axis=1, inplace=True)

In [ ]:
# Drop rows with null values
df.dropna(inplace=True)

In [ ]:
# Temp fill numerical null values
for col in numerical_vals:
    df[col].fillna(df[col].mean(), inplace=True)

In [ ]:
# Temp fill categorical null values
df[col].fillna('Missing')

In [ ]:
# ...other filling methods...
# method : {‘backfill’, ‘bfill’, ‘pad’, ‘ffill’}

In [ ]:
# Look at any duplicate values
dups = df[df.duplicated()]
dups

In [ ]:
# Drop duplicates
df.drop_duplicates(inplace=True)

In [ ]:
# Check for duplicate identifiers
dup_ids = pd.concat(x for _, x in df.groupby('_') if len(x) > 1).sort_values('_')
dup_ids

In [ ]:
# Remove any duplicate identifiers
# Pandas will keep the first value, and drop all the following rows
df.drop_duplicates('_', inplace=True)

In [ ]:
# Sort data to make sure you drop the intended row
df.sort_values(['_','_'], inplace=True)

### Summary Stats

In [ ]:
cols = df.columns.values
numerical_vals = df.select_dtypes(exclude=['object', 'bool'])                                     
categorical_vals = df.select_dtypes(include=['object', 'bool'])

In [ ]:
# Look at summary statistics for continous values
df.describe()

In [ ]:
# Check for collinearity between variables
c = df.corr().round(4).abs()
start = int(c.shape[0])
c.unstack().sort_values(ascending=False)[start:start+10] # top 5
# c.to_csv('../data/correlation.csv')

### Visualization

In [ ]:
!mkdir plots

seaborn plots...

In [ ]:
# Bar graphs of individual categorical columns
for i,col in enumerate(categorical_vals):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.set_title(col)
    c = sns.countplot(x=df[col], ax=ax);
    c.set_xticklabels(c.get_xticklabels(), rotation=45)
    plt.savefig('../plots/bargraph_{}'.format(col))

In [ ]:
# Boxplots of individual numerical columns
for i,col in enumerate(numerical_vals):
    fig = plt.figure(figsize=(7,7))
    ax = fig.add_subplot(111)
    sns.violinplot(x=df[col], orient='v', ax=ax)
    text = '75th Percentile: {}\nMedian: {}\n25th Percentile: {}'.format(np.percentile(df[col], 75),\
            np.median(df[col]),np.percentile(df[col], 25))
    at = AnchoredText(text, prop=dict(size=15), frameon=True, loc=1)
    ax.add_artist(at)
    plt.savefig('../plots/violinplot_{}'.format(col))

In [ ]:
# Stripplot of target (continuous) by all object columns values
target = _ # continuous
for i,col in enumerate(categorical_vals):
    if col != target: 
        fig = plt.figure(figsize=(7,7))
        ax = fig.add_subplot(111)
        sns.stripplot(x=df[col], y=df[target], orient='v', ax=ax)
        plt.savefig('../plots/stripplot_{}'.format(col))

In [ ]:
# Scattermatrix of all numerical columns
sns.pairplot(df[numerical_vals])
plt.savefig('../plots/scattermatrix')

pandas plots...  
http://pandas.pydata.org/pandas-docs/stable/visualization.html  
df.plot.area     df.plot.barh     df.plot.density  df.plot.hist     df.plot.line     df.plot.scatter
df.plot.bar      df.plot.box      df.plot.hexbin   df.plot.kde      df.plot.pie

In [ ]:
# Histogram 
df[num_col].plot.hist(bins=20)
df[cat_col].value_counts().plot(kind='bar')

In [ ]:
# Line
df[num_col].plot(logx=False, logy=False)
df[num_col].plot(x_compat=True, secondary_y=False)
df.plot(subplots=True, figsize=(6, 6))

In [ ]:
# Bar
df[numerical_vals].plot.bar(yerr=errors) # errors (std dev) for error bars 

In [ ]:
# Box
color = dict(boxes='DarkGreen', whiskers='DarkOrange', medians='DarkBlue', caps='Gray')
df[numerical_vals].plot.box(color=color, sym='r+', vertical=True)

df.groupby(cat_col).boxplot()

df.boxplot(by='X') # stratified by X
df.boxplot(column=['Col1','Col2'], by=['X','Y'])

In [ ]:
# Scatter and scatter matrix
df[num_col].plot.scatter(x='', y=df[target], c=color_values, s=size)
pd.plotting.scatter_matrix(df[numerical_vals], alpha=0.2, figsize=(6, 6), diagonal='kde')

In [ ]:
# Verify that target data is normally distributed
df[target].plot.kde
# If not, transform with log or sqrt
df[target] = np.log(df[target])

### Aggregate and groupby

In [ ]:
df["col"].value_counts() # count number of each unique value

In [ ]:
df.groupby(target).get_group(True)

In [ ]:
# Cross-tabulation of feature vs target
pd.crosstab(df.col, df.target)

In [ ]:
# Turn continuous data into categorical data 
bin_series = pd.cut(df[col], bins=np.arange(1, 10))
bin_series.name = ''
pd.concat([df, bin_series], axis=1)

In [ ]:
# Pivot table
print(cols)
pd.pivot_table(df, values=col, index=target, columns=col, aggfunc=lambda x: len(x))

In [ ]:
df.groupby(col).mean() 
# .std() Standard deviation ouses n-1, not N, by default

In [ ]:
agg = df.agg(mean)
agg.unstack(level = 'column') # Takes column agg and moves from rows to columns

In [ ]:
# Map ranked values in list {‘low’:1, ‘medium’:2, ‘high’:3}
pd.Series(pd.Categorical(values, categories=['low', 'medium', 'high'], ordered=True))

check for normality, imbalanced classes, outliers, high leverage points

### Formatting data

In [ ]:
# Apply formatting where appropriate
df['Names'] = df.Names.apply(lambda x: x.lower())
pd.series.map()
df.applymap()

In [ ]:
# Review unique values
df[col].unique()
df[col].nunique()

In [ ]:
# Create dummy variables
df = pd.get_dummies(df.col)
df = pd.get_dummies(df, columns=['col1', 'col2'], drop_first=True, dummy_na=True, prefix='dum')

In [ ]:
# Force object coded columns to numerical values
df[col] = pd.to_numeric(df.col, errors = 'coerce')

### Join dataframes, add/remove columns

In [ ]:
# Join by appending rows/columns based on a given axis
pd.concat([df1,df2,series], axis=1)
# Merge on common columns
pd.merge(df1, df2, on=’column’)
# Join on common indices
joined = df1.join(df2)

In [ ]:
# Create an empty column 
df['empty col'] = np.nan

In [ ]:
# Append a new row
df.append(new, ignore_index=True)

In [ ]:
# Delete or drop or remove a column
del df[col]
df.drop([col1,col2], inplace=True, axis=1)